## KR260 HLS QPSK Demod


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [67]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

#### Load Overlay

In [81]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time
import kria
import timestamp_regmap
import led_regmap


# Load the overlay
overlay = Overlay('/home/root/jupyter_notebooks/kr260_hls_qpsk_demod/kr260_hls_qpsk_demod.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
qpsk_demod   = overlay.hls_qpsk_demod_0

##### Print out the register map for DMA and HLS Gain Blocks

In [ ]:
overlay.ip_dict

In [ ]:
qpsk_demod.register_map

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [73]:
print("FPGA Build Timestamp:  " + kria.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/2/16 16:59:51


##### Toggle USER_LED[1:0] on/off

In [ ]:
for i in range(10):
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x1)
    time.sleep(0.25)
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x2)
    time.sleep(0.25)
led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x0)

### Configure the QPSK Demod Control Register

Want to enable auto restart, so the FPGA block will always be read to receive samples


In [82]:
qpsk_demod.register_map.CTRL.AUTO_RESTART=1
qpsk_demod.register_map.CTRL.AP_START=1
qpsk_demod.register_map.CTRL

Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0)

### Open Modulated Data Samples and Convert to INT32

In [83]:
mod_samps = []
with open('modulatedData_short.dat') as mod:
    mod_samps = mod.readlines()
mod_samps = [line.rstrip('\n') for line in mod_samps]

mod_samps = np.array(mod_samps)
mod_samps = mod_samps.astype('float64')*(2**28)
mod_samps = mod_samps.astype(np.int32)

print("Number of Samples Read: " + str(len(mod_samps)))
plot_time(mod_samps)

Number of Samples Read: 512


Loading BokehJS ...

### DMA Transfer the Sample buffer from ARM Processor to the HLS QPSK Demodulator

In [84]:
# 16 Samples Per Symbol, therefore create read buffer /16
SamplesPerSym = 16
NumSamples = len(mod_samps)
NumSymbols = int(NumSamples/SamplesPerSym/4)
print("NumSamples = " + str(NumSamples))
print("NumSymboles = " + str(NumSymbols))

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(NumSamples,), dtype=np.int32)
out_buffer = allocate(shape=(NumSymbols,), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,mod_samps)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
print("Done tranfer in_buffer")
dma.recvchannel.transfer(out_buffer)
print("Done transfer out_buffer")
#dma.sendchannel.wait()
print("Done send channel wait")
#dma.recvchannel.wait()
print("Done receive channel wait")

stop_time = time.time()
hw_exec_time = stop_time-start_time

print('DMA Transfer Execution Time  :',hw_exec_time,' sec')

# Plot to the notebook
#plot_time(t,samples,2000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()
out_buffer.close()



NumSamples = 512
NumSymboles = 8
Done tranfer in_buffer
Done transfer out_buffer
Done send channel wait
Done receive channel wait
DMA Transfer Execution Time  : 0.0017931461334228516  sec
